# Modélisation d’un réseau BT – Trey
Par M. Buchser & M. Joye

Réalisé lors du semestre n°1 de 2024-2025



### Code of the work done

Import of librairies

In [281]:
import numpy as np
import pandas as pd
import pandapower as pp
import pp_heig_plot as pp_plot
import pp_heig_simulation as pp_sim
from datetime import time

Import the net from the Excel file

In [282]:
net_file_path = "data/trey_power_network.xlsx"
net = pp_sim.load_net_from_xlsx(file_path=net_file_path)
# net

Control that everything is okay

In [283]:
# net.bus
# net.res_line
# net.ext_grid

Plot the net

In [284]:
plot = pp_plot.plot_power_network(net, plot_title="Network of Trey")

Test power flow - plot with constant flow

In [285]:
pp.runpp(net)
pp_plot.plot_powerflow_result(net=net, plot_title="Network of Trey")

In [286]:
# net.res_bus

In [287]:
# net.res_line

In [288]:
# net.res_trafo

2.1.2.1 Identification and modeling of load curves

In [289]:
# Import datas from CSV file
consumer_file_path = "data/liste_des_batiments_vf.csv"
consumer_data = pd.read_csv(consumer_file_path, encoding="latin1")

# Control
# consumer_data

2.1.2.2 Estimation of annual consumption using floor space

In [290]:
# COP estimation for heat cumsuption with heat pump
COP = 4

# Create a mapping dictionary for consumption (SIA standard)
consumption_sia = {
    "one_housing": {
        "electricity": 17.8 + 0 + 4.2,
        "heating": 23.4,
        "hot_water": 13.5,
        "special_heating": 23.4 / COP,
        "special_hot_water": 13.5 / COP,
    },  # Individual housing
    "multi_housing": {
        "electricity": 21.6 + 0 + 4.2,
        "heating": 14.2,
        "hot_water": 16.9,
        "special_heating": 14.2 / COP,
        "special_hot_water": 16.9 / COP,
    },  # Collective housing
    "farm": {
        "electricity": 6.3 + 0 + 10.5,
        "heating": 11.5,
        "hot_water": 0.9,
        "special_heating": 11.5 / COP,
        "special_hot_water": 0.9 / COP,
    },  # Agricultural building (assimilated to warehouse)
    "church": {
        "electricity": 4.6 + 0 + 20.9,
        "heating": 6.4,
        "hot_water": 7.3,
        "special_heating": 6.4 / COP,
        "special_hot_water": 7.3 / COP,
    },  # Ecclesiastical building (assimilated to performance hall)
    "industial": {
        "electricity": 16.8 + 50 + 20.5,
        "heating": (9.0 + 10.7) / COP,
        "hot_water": 2.4 / COP,
    },  # Industrial building (heavy work)
}


# Function to estimate electricity consumption
def estimate_consumption(row):
    building_type = row["building_class"]
    surface = row["Empreinte au sol [m2]"]

    if building_type in consumption_sia:
        conso = consumption_sia[building_type]
        # Calculate electricity consumption (sum of appliances, installations, and lighting)
        electricity_consumption = conso["electricity"] * surface

        # Add thermal energy if heated electrically
        heating_consumption = 0
        if row["heat_source_1"] == "electricity":
            heating_consumption += conso["heating"] * surface
        elif row["heat_1"] == "PAC_1":
            heating_consumption += conso["special_heating"] * surface
            if row["heatwater_source_1"] == "unknown":
                heating_consumption += conso["special_hot_water"] * surface
        elif row["building_class"] == "industial":
            heating_consumption += (conso["heating"] + conso["hot_water"]) * surface
        if row["heatwater_source_1"] == "electricity":
            heating_consumption += conso["hot_water"] * surface

        return electricity_consumption + heating_consumption
    else:
        print(f"Building class {building_type} is missing.")


# Apply the function to each row
consumer_data["Estimated_Electricity_Consumption_kWh"] = consumer_data.apply(
    estimate_consumption, axis=1
)

In [291]:
# # Display the results (verification)
# consumer_data[
#     [
#         "Cabinet",
#         "building_class",
#         "Empreinte au sol [m2]",
#         "Estimated_Electricity_Consumption_kWh",
#     ]
# ]

2.1.2.3 Definition of two scenarios of load curve

In [292]:
# Specify the file path
file_path = "data/Load_curves.xlsx"

# List of sheets to read
sheet_names = ["one_housing", "multi_housing", "farm", "church", "industial"]

# Dictionary to store data from each sheet
load_curves_df = {}

# Read the specified sheets
for sheet in sheet_names:
    load_curves = pd.read_excel(file_path, sheet_name=sheet)

    # Store the cleaned data
    load_curves_df[sheet] = load_curves

    # Normalize the 'winter' and 'summer' columns for each building type
for sheet, df in load_curves_df.items():
    # Calculate the correction factor for the 'winter' and 'summer' columns
    winter_corr_fact = df["winter"].sum() * 365 / (4 * 1000 * 1000)
    summer_corr_fact = df["summer"].sum() * 365 / (4 * 1000 * 1000)

    # Normalize the columns
    df["winter"] = df["winter"] / winter_corr_fact
    df["summer"] = df["summer"] / summer_corr_fact

    # Store the normalized DataFrame back into the dictionary
    load_curves_df[sheet] = df

In [293]:
# Verification of the 1000 kWh/year value
# for sheet, df in load_curves_df.items():
#     # Display the new total for verification
#     new_winter_sum = df["winter"].sum() * 365 / (4 * 1000)
#     new_summer_sum = df["summer"].sum() * 365 / (4 * 1000)
#     print(f"{sheet} - Normalized winter total: {new_winter_sum:.2f} kWh/year")
#     print(f"{sheet} - Normalized summer total: {new_summer_sum:.2f} kWh/year")

In [294]:
# Display a preview of the data with the sheet name as a title (verification)
for sheet, df in load_curves_df.items():
    print("=" * 30)  # Separator line
    print(f"Data from sheet: {sheet}")  # Display the sheet name
    print("=" * 30)
    print(df.head())  # Display the first rows of the dataframe
    print("\n")  # Add spacing for better readability

Data from sheet: one_housing
       time     winter     summer
0  00:15:00  67.199932  96.092003
1  00:30:00  64.732138  88.796306
2  00:45:00  62.549089  82.460570
3  01:00:00  60.081295  76.700809
4  01:15:00  56.474519  71.133041


Data from sheet: multi_housing
       time     winter     summer
0  00:15:00  67.199932  96.092003
1  00:30:00  64.732138  88.796306
2  00:45:00  62.549089  82.460570
3  01:00:00  60.081295  76.700809
4  01:15:00  56.474519  71.133041


Data from sheet: farm
       time     winter     summer
0  00:15:00  63.503583  58.889415
1  00:30:00  63.223008  58.342115
2  00:45:00  63.410058  58.013736
3  01:00:00  63.410058  57.575896
4  01:15:00  62.942432  56.590757


Data from sheet: church
       time     winter     summer
0  00:15:00  54.203502  78.060329
1  00:30:00  54.444406  72.041658
2  00:45:00  55.006516  67.117292
3  01:00:00  54.685311  63.196037
4  01:15:00  52.517170  60.004318


Data from sheet: industial
       time     winter      summer
0  00:15

Definition of the two consumption curves on each cabinet

In [295]:
# Dictionary to store the load curves for each cabinet
cabinet_load_curves_df = {}

# Loop through each row in consumer_data
for index, row in consumer_data.iterrows():
    building_class = row["building_class"]
    cabinet = row["Cabinet"]
    estimated_consumption = row["Estimated_Electricity_Consumption_kWh"]

    # Find the corresponding load curve for the building class
    if building_class in load_curves_df:
        load_curve = load_curves_df[building_class].copy()

        # Calculate the scaling factor
        scaling_factor = estimated_consumption / 1000

        # Scale the summer and winter curves
        load_curve["winter"] *= scaling_factor
        load_curve["summer"] *= scaling_factor

        # Add this load curve to the dictionary under the cabinet's name
        if cabinet in cabinet_load_curves_df:
            # Only adding the curves, not the time
            cabinet_load_curves_df[cabinet]["winter"] += load_curve["winter"]
            cabinet_load_curves_df[cabinet]["summer"] += load_curve["summer"]

        else:
            # Create a new DataFrame for this cabinet
            cabinet_load_curves_df[cabinet] = load_curve

In [296]:
# Display the data for a specific cabinet (verification)
for cabinet, df in cabinet_load_curves_df.items():
    print(f"\nData for Cabinet: {cabinet}")
    print(df.head())


Data for Cabinet: CDBT016055
       time       winter       summer
0  00:15:00  1658.642153  2371.762026
1  00:30:00  1597.731566  2191.688186
2  00:45:00  1543.849123  2035.308272
3  01:00:00  1482.938535  1893.144714
4  01:15:00  1393.915369  1755.719941

Data for Cabinet: CDBT012139
       time      winter      summer
0  00:15:00  520.893550  744.847549
1  00:30:00  501.764691  688.295687
2  00:45:00  484.843008  639.184860
3  01:00:00  465.714149  594.538653
4  01:15:00  437.756585  551.380653

Data for Cabinet: CDBT004774
       time       winter       summer
0  00:15:00  1112.053029  1590.171298
1  00:30:00  1071.214924  1469.439011
2  00:45:00  1035.088907  1364.592552
3  01:00:00   994.250801  1269.277589
4  01:15:00   934.564339  1177.139792

Data for Cabinet: CDBT901452
       time       winter       summer
0  00:15:00  3866.395108  5528.720632
1  00:30:00  3724.408847  5108.957627
2  00:45:00  3598.805616  4744.426597
3  01:00:00  3456.819355  4413.034751
4  01:15:00  3249.

Adding of the solar production on the cabinet curves

In [297]:
# File path for the solar production curves
solar_file_path = "data/Solar_production_curves.xlsx"

# Read all sheet names from the solar production file
solar_sheet_names = pd.ExcelFile(solar_file_path).sheet_names

# Dictionary to store solar production curves
solar_curves_df = {}

# Read solar production curves for each cabinet
for sheet in solar_sheet_names:
    # Read the sheet
    solar_data = pd.read_excel(solar_file_path, sheet_name=sheet)

    # Rename columns for consistency
    solar_data.rename(
        columns={
            "Temps (heure:min)": "time",
            "Énergie [kWh] hiver": "winter",
            "Énergie [kWh] été": "summer",
        },
        inplace=True,
    )

    # Convert 'time' column to match load curve format
    solar_data["time"] = pd.to_datetime(solar_data["time"], format="%H:%M:%S").dt.time

    # Store the solar curve in the dictionary
    solar_curves_df[sheet] = solar_data

# Dictionary to store updated load curves
updated_cabinet_load_curves_df = {}

# Subtract solar production curves from load curves
for cabinet, load_curve in cabinet_load_curves_df.items():
    if cabinet in solar_curves_df:
        # Get the solar production curve for this cabinet
        solar_curve = solar_curves_df[cabinet]

        # Merge load and solar curves on 'time' to align them
        merged_curve = pd.merge(
            load_curve, solar_curve, on="time", suffixes=("_load", "_solar")
        )

        # Subtract solar production from the load curves
        merged_curve["summer"] = (
            merged_curve["summer_load"] - merged_curve["summer_solar"]
        )
        merged_curve["winter"] = (
            merged_curve["winter_load"] - merged_curve["winter_solar"]
        )

        # Drop unnecessary columns and retain the updated load curve
        cabinet_load_curves_df[cabinet] = merged_curve[["time", "summer", "winter"]]
        updated_cabinet_load_curves_df[cabinet] = merged_curve[
            ["time", "summer", "winter"]
        ]
    else:
        print(f"No solar production curve found for cabinet: {cabinet}")

No solar production curve found for cabinet: CDBT016055
No solar production curve found for cabinet: CDBT012139
No solar production curve found for cabinet: CDBT004774
No solar production curve found for cabinet: CDBT901452
No solar production curve found for cabinet: STMT003438
No solar production curve found for cabinet: CDBT901604
No solar production curve found for cabinet: CDBT004760
No solar production curve found for cabinet: CDBT004764
No solar production curve found for cabinet: CDBT003746
No solar production curve found for cabinet: CDBT900784


In [298]:
# Display the updated load curve for a specific cabinet (verification)
# for cabinet, df in updated_cabinet_load_curves_df.items():
#     print(f"\nUpdated Load Curve for Cabinet: {cabinet}")
#     print(df)
#     print(f"\nSolar Production Curve for Cabinet: {cabinet}")
#     print(solar_curves_df[cabinet])

Cabinet curve save in the "power_profile.xlsx" file

Import the power profile from the Excel file

In [299]:
# power_profile_file_path = "data/power_profile.xlsx"
# power_profile = pp_sim.load_power_profile_form_xlsx(file_path=power_profile_file_path)

Apply the profile and the net

In [300]:
# apply = pp_sim.apply_power_profile(power_profile)

Simulation

In [301]:
# pp_sim.run_time_simulation(net)